In [1]:
from viz_classes import BarChartCreator, LineChartCreator, PieChartCreator
from dash import Dash, html, Input,Output,State,callback,dcc
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as go
import plotly.graph_objects as go
import datetime

In [2]:
df = pd.read_parquet("summary_all.parquet")
df.head()

,building_no,Zone_name,Datetime,Season,Faulty,Fan_status,Fan_time_diff,Predicted
0,1,ActEast,2023-03-23,2,True,None,34200.0,31053.130335
1,1,ActEast,2023-03-24,2,True,None,86640.0,52959.389325
2,1,ActEast,2023-03-25,2,True,None,86400.0,72425.248406
3,1,ActEast,2023-03-26,2,True,None,86400.0,56703.078787
4,1,ActEast,2023-03-27,2,True,None,86400.0,42406.888444


In [3]:
initial_bar = BarChartCreator(df,1)
initial_bar.fig

In [4]:
df_1 = df[df['building_no'] == 1]
len(df_1)

15370

In [16]:
initial_bar = BarChartCreator(df_1,1)
initial_pie = PieChartCreator(df_1,1,'Acu101')
initial_line = LineChartCreator(df_1,1,'Acu101')
app = Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Row([
        html.H1("Building 1 Savings"),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id="bar-graph",figure=initial_bar.fig,style={"height":"300px"})
        ],width=9),
        dbc.Col([
            dcc.Graph(id="pie-chart",figure=initial_pie.fig,style={"height":"300px"})
        ],width = 3)
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                    options=[
                        {'label':'Daily','value':'day'},
                        {'label':'Weekly','value':'week'},
                        {'label':'Monthly','value':'month'}
                    ],
                    value="day",
                    id="agg-type",
            )
        ],width=9),
        dbc.Col([
            dcc.DatePickerRange(
                min_date_allowed=df_1.Datetime.min(),
                max_date_allowed=df_1.Datetime.max(),
                start_date=df_1.Datetime.min(),
                end_date=df_1.Datetime.max(),
                id='date-picker-range',
                clearable=True,
                updatemode='bothdates',
                display_format="MMM Do, YY",
                start_date_placeholder_text="MMM Do, YY",
                end_date_placeholder_text="MMM Do, YY"
            )
        ])
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id="line-chart",figure=initial_line.fig,style={"height":"300px"})
        ])
    ]),
],fluid=True)

@app.callback(
    [Output("line-chart","figure"),
     Output("pie-chart","figure")],
    [Input("agg-type","value"),
     Input('bar-graph','clickData'),
     Input("date-picker-range",'start_date'),
     Input("date-picker-range",'end_date')],
)
def update(type_agg,clickData,start_date,end_date):
    start = [int(x) for x in start_date.split("-")]
    end = [int(x) for x in end_date.split("-")]
    zone = clickData['points'][0]['x'] if clickData else "Acu101"
    date_range_var=[datetime.date(start[0],start[1],start[2]),datetime.date(end[0],end[1],end[2])]
    updated_line = LineChartCreator(df_1,1,zone,date_range=date_range_var,agg=type_agg).fig
    updated_pie = initial_pie.update_fig(df_1,1,zone,date_range_var)
    return updated_line, updated_pie

if __name__ == "__main__":
    app.run(debug=True,port=8000)